In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 

In [7]:
#Read data of sensors
sensorData = pd.read_excel('Data/Environement/K=3/SensorInformation.xls')
sensorCoordinate = []

In [28]:
#Sensors coordinate 
for row in sensorData.index:
    coordinate = (sensorData['X'][row] ,sensorData['Y'][row] )
    sensorCoordinate.append(coordinate)

In [32]:
#Cluster Information 
clusterData = pd.read_excel('Data/Environement/K=3/ClustersInformations.xls')
ClusterInformation = []

In [35]:
#Cluster coordinate 
for row in clusterData.index:
    coordinate = (clusterData['X'][row] ,clusterData['Y'][row] )
    ClusterInformation.append(coordinate)


In [41]:
# Normlized Data of the Mission 
nomlizedMissionData = pd.read_excel("NormlizedData.xls")
missionInformationSubPath = []
missionInformationScores =  []
for row in nomlizedMissionData.index:
    subPath = (nomlizedMissionData['Cluster_ID'][row] , nomlizedMissionData['Cluster_DESTINATION'][row])
    scores  =(float(nomlizedMissionData['TIME_TRAVEL_CONSUMPTION'][row]),float(nomlizedMissionData['HOVERING_TIME'][row]) , float(nomlizedMissionData['ENERGY_HOVER'][row])  , float(nomlizedMissionData['TRAVEL_ENERGY_COST'][row]))
    missionInformationSubPath.append(subPath)
    missionInformationScores.append(scores)

In [42]:
# Starting with gurobi , Match subpath with there scores 
import gurobipy as gp 
from gurobipy import GRB 

In [44]:
dataDict = {}
for cluster in range(len(missionInformationSubPath)):
    dataDict.update({missionInformationSubPath[cluster] : missionInformationScores[cluster]})

In [54]:
#Match subpath with there scores  
combinations , timeTravel , hoveringTime , energyTravel , hoveringEnergy  = gp.multidict(dataDict)

In [60]:
numberOfCluster = 14

In [66]:
# The decision variable of the subPath it's equal to 1 if the cluster i is connected with the cluster j 
#Initialization of the model 
model  = gp.Model('UAV_TimeEnergy_Minimization')

Restricted license - for non-production use only - expires 2023-10-25


In [ ]:
# Create the decision variable for the UAV_TimeEnergy_Minimization 
# combinations contain the cluster i and the cluster j i--j or a subPath 
